In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def locate(t,x,g=0):
    if x[g]<=t and t<=x[g+1]:
        return g
    left = 0
    right = len(x)-1
    while right>left+1:
        mid = (right+left)//2
        if t >= x[mid]:
            left = mid
        else:
            right = mid
    return left

def pwC(x,y,s):
    s = np.array(s)
    y = np.array(y)
    dx = np.diff(x)
    dy = np.diff(y)
    yp = dy/dx
    a = y[:-1]
    b = s[:-1]
    c = (yp-s[:-1])/dx
    d = (s[:-1]+s[1:]-2*yp)/(dx*dx)
    return [a,b,c,d]

def pwCEval(t,x,coef):
    ta = np.array(t)
    a,b,c,d = coef
    result = np.empty(ta.size)
    i = 0
    for tk,k in zip(ta,range(ta.size)):
        i = locate(tk,x,i)
        result[k] = d[i]*(tk-x[i+1])+c[i]
        result[k] = result[k]*(tk-x[i])+b[i]
        result[k] = result[k]*(tk-x[i])+a[i]
    return result

In [3]:
def MyCubicSpline(x,y,der=3,muL=0,muR=0):
    y = np.array(y)
    dx = np.diff(x)
    dy = np.diff(y)
    yp = dy/dx
    A = np.diag(2*(dx[1:]+dx[:-1]))+np.diag(dx[2:],-1)+np.diag(dx[:-2],1)
    b = 3*(dx[:-1]*yp[1:]+dx[1:]*yp[:-1])
    if der == 1:
        b[0]  -= dx[1]*muL
        b[-1] -= dx[-2]*muR
        s = np.linalg.solve(A,b)
        s = np.hstack((muL,s,muR))
    elif der == 2:
        bd1 = (6*yp[0]-muL*dx[0])/4
        b[0] -= dx[1]*bd1
        A[0,0] -= dx[1]/2
        bdn = (6*yp[-1]-muL*dx[-1])/4
        b[-1] -= dx[-2]*bdn
        A[-1,-1] -= dx[-2]/2
        s = np.linalg.solve(A,b)
        s = np.hstack((bd1-s[0]/2,s,bdn-s[-1]/2))
    elif der==3:
        q = dx[0]*dx[0]/dx[1]
        b[0] -= 2*(yp[0]*dx[1]-yp[1]*q)
        A[0,0] += q-dx[1]
        A[0,1] += q
        q = dx[-1]*dx[-1]/dx[-2]
        b[-1] -= 2*(yp[-1]*dx[-2]-yp[-2]*q)
        A[-1,-1] += q-dx[-2]
        A[-1,-2] += q
        s = np.linalg.solve(A,b)
        sL = 2*yp[0]-s[0]+((dx[0]/dx[1])**2)*(s[0]+s[1]-2*yp[1])
        sR = 2*yp[-1]-s[-1]+((dx[-1]/dx[-2])**2)*(s[-1]+s[-2]-2*yp[-2])
        s = np.hstack((sL,s,sR))
    else:
        print('d must be one of 1, 2 or 3')
        return None
    a = y[:-1]
    b = s[:-1]
    c = (yp-s[:-1])/dx
    d = (s[:-1] + s[1:]-2*yp)/(dx*dx)
    return a,b,c,d